In [1]:
import pandas as pd
import glob, os

In [2]:
rename_fields = {
    'Squadra': 'team',
    'Rf': 'goals',
    'Tiri': 'total_shots',
    'Tiri.1': 'shots_on_target',
    'Rigori': 'goals_on_penalty',
    'Rig T': 'total_penalties',
    'Compl': 'completed_passings',
    'Tent': 'total_passings',
    'Angoli': 'corners',
    'Poss.': 'percentage_possession',
    'Falli': 'fouls',
    'Amm.': 'yellow_cards',
    'Esp.': 'red_cards',
    'Data': 'date',
    'Girone': 'matchday',
}

Per ciascuna squadra, per ciascuna partita, devo calcolare il valore "previsto" che è dato dalla somma di tutti i valori nelle partite precedenti. 
Il codice sottostante permette di fare questo.

I file relativi alle statistiche delle varie squadre sono salvati in una cartella. Prendo tutti i file, li leggo e calcolo le medie

In [4]:
stats_of_all_teams = {}
for file in sorted(os.listdir("SerieA/Season21_22/Matches/")):
    if file.endswith(".csv"):
        team_name = file.replace('.csv', '')
        team_stats = pd.read_csv("SerieA/Season21_22/Matches/"+file, index_col=0)
        team_stats = team_stats.rename(columns=rename_fields) #rinomino le colonne per avere una nomenclatura 
        #team_stats = team_stats.drop(columns=['Stadio', 'Avversario', 'Rs', 'team', 'date', 'matchday', 'Risultato'])
        team_stats = team_stats.drop(columns=['Stadio', 'Avversario', 'Rs'])
        dataset = team_stats.drop(columns=['team', 'date', 'matchday', 'Risultato'])

        for single_col in dataset.columns:
            team_stats[single_col] = team_stats[single_col].astype(float) #converto la colonna in float se no vengono approssimati i valori per difetto
            team_column = dataset[single_col]
            
            for index, value in enumerate(team_column.values):
                sum = value if index == 0 else 0
                for from_0_to_index in range(index): 
                    sum += dataset.iloc[from_0_to_index][single_col]
                
                mean = (sum / index) if index > 0 else sum
                team_stats.at[index, single_col] = mean
        
        team_stats['date'] = pd.to_datetime(team_stats['date'], format='%d-%m-%Y') #devo convertire la data altrimenti mi dà problemi quando la lego 
        team_stats.to_csv("SerieA/Season21_22/AvgStats2/"+file)